In [0]:
#Step 1 - Read the JSON file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

In [0]:

name_schema = StructType(fields=[StructField("forename", StringType(), True),
                                 StructField("surname", StringType(), True)
  
])

In [0]:

drivers_schema = StructType(fields=[StructField("driverId", IntegerType(), False),
                                    StructField("driverRef", StringType(), True),
                                    StructField("number", IntegerType(), True),
                                    StructField("code", StringType(), True),
                                    StructField("name", name_schema),
                                    StructField("dob", DateType(), True),
                                    StructField("nationality", StringType(), True),
                                    StructField("url", StringType(), True)  
])

In [0]:
drivers_df = spark.read.json("dbfs:/FileStore/drivers.json" , schema = drivers_schema)

In [0]:
#Step 2 - Rename columns and add new columns
#driverId renamed to driver_id
#driverRef renamed to driver_ref
#ingestion date added
#name added with concatenation of forename and surname

In [0]:
from pyspark.sql.functions import col, concat, lit , current_timestamp

In [0]:
drivers_with_ingestion_date_df = drivers_df.withColumn("ingestion_date",current_timestamp())

In [0]:
drivers_with_columns_df = drivers_with_ingestion_date_df.withColumnRenamed("driverId", "driver_id") \
                                    .withColumnRenamed("driverRef", "driver_ref") \
                                    .withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname"))) 

In [0]:
#Step 3 - Drop the unwanted columns
#name.forename
#name.surname
#url

In [0]:
drivers_final_df = drivers_with_columns_df.drop(col("url"))

In [0]:
#Step 4 - Write to output to processed container in parquet format

In [0]:
drivers_final_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.drivers") 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-2747915330769742>:1
----> 1 drivers_final_df.write.parquet("dbfs:/FileStore/Formula1/processed/f1_processed.drivers")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py:1321, 

In [0]:
drivers_final_df.show(truncate=False)

+---------+----------+------+----+------------------+----------+-----------+-----------------------+
|driver_id|driver_ref|number|code|name              |dob       |nationality|ingestion_date         |
+---------+----------+------+----+------------------+----------+-----------+-----------------------+
|1        |hamilton  |44    |HAM |Lewis Hamilton    |1985-01-07|British    |2024-04-22 18:07:10.948|
|2        |heidfeld  |null  |HEI |Nick Heidfeld     |1977-05-10|German     |2024-04-22 18:07:10.948|
|3        |rosberg   |6     |ROS |Nico Rosberg      |1985-06-27|German     |2024-04-22 18:07:10.948|
|4        |alonso    |14    |ALO |Fernando Alonso   |1981-07-29|Spanish    |2024-04-22 18:07:10.948|
|5        |kovalainen|null  |KOV |Heikki Kovalainen |1981-10-19|Finnish    |2024-04-22 18:07:10.948|
|6        |nakajima  |null  |NAK |Kazuki Nakajima   |1985-01-11|Japanese   |2024-04-22 18:07:10.948|
|7        |bourdais  |null  |BOU |Sébastien Bourdais|1979-02-28|French     |2024-04-22 18:0